# Hello QMI

Welcome to your QMI! Your Quantum Machine Image is a virtual development and execution environment for quantum programming. It is hosted in close physical proximity to the QPUs in Rigetti’s quantum data center. This integrated platform eliminates the network latency found in web API access models, allowing you to iterate faster. It comes with a pre-configured Jupyter server that is a great way to get started writing quantum programs, and running them on the QVM and the QPU. Don't forget to also try connecting to your QMI via `ssh` (Secure Shell), and moving files to and from your QMI using Secure Copy (`scp`). For more information on this, check out our [documentation](https://www.rigetti.com/qcs/docs/getting-started-with-your-qmi#qmi).

In this beginner's example, we are going to run a simple `hello_qmi()`. This function composes a simple quantum program, compiles it, and then runs it on the target device of your choice: either the QVM or a QPU. This function is also located in the `hello_qmi.py` file in the home folder of your QMI at `/home/forest/hello_qmi.py`. You can browse to this file directly in Jupyter, or find it by `ssh`ing into your QMI.

## Imports

To import `hello_qmi`, we must include in our path the folder in our QMI where it's located: one folder up from our current directory, or `..`.

In [1]:
import math
import os
import sys

from pyquil import Program, get_qc
from pyquil.api import QVM
from pyquil.api._devices import list_lattices
from pyquil.gates import RX, MEASURE
from pyquil.quil import Pragma

## Lattice-related helper functions

In [2]:
def query_device(device_name):
    """
    Try to query the device from QCS. Return the lattice dict if it exists,
    or None otherwise.
    """
    lattices = list_lattices()
    return lattices.get(device_name)


def get_active_lattice():
    """
    Try to query which lattice we're engaged to from QCS. Returns the lattice
    name if available, otherwise None.
    """
    from rpcq import Client
    from pyquil.api._config import PyquilConfig
    try:
        qcc = Client(endpoint=PyquilConfig().qpu_compiler_url, timeout=1)
        return qcc.call("get_config_info")["lattice_name"]
    except:
        return None

## Run against the QVM Simulator

The function `hello_qmi` takes an optional argument naming the device to target (the QVM or a QPU). If no argument is provided, the default device is a `9q-generic-qvm`.

In [3]:
def hello_qmi(device_name="9q-generic-qvm", shots=5):
    """
    Get acquainted with your quantum computer by asking it to perform a simple
    coin-toss experiment. Involve 3 qubits in this experiment, and ask each one
    to give `shots` many results.

    :param device_name: The name of a quantum computer which can be retrieved
                        from `pyquil.api.get_qc()`. To find a list of all
                        devices, you can use `pyquil.api.list_devices()`.
    :param shots:       The number of times to run the Quil program steps
    """
    # Initialize your Quil program
    program = Program()
    # Allow the compiler to re-index to use available qubits, if necessary.
    program += Pragma('INITIAL_REWIRING', ['"GREEDY"'])
    device = query_device(device_name)
    if device is not None:
        # device_name refers to a real (QPU) device, so let's construct
        # the program from the device's qubits.
        readout = program.declare('ro', 'BIT', len(device['qubits']))
        for qubit in device['qubits'].values():
            program += RX(math.pi / 2, qubit)
        for idx, qubit in enumerate(device['qubits'].values()):
            program += MEASURE(qubit, readout[idx])
    else:
        # device_name refers to a non-real (QVM) device, so let's construct
        # the program from arbitrary qubits, e.g. 0, 1, and 2

        # Declare 3 bits of memory space for the readout results of all three qubits
        readout = program.declare('ro', 'BIT', 3)
        # For each qubit, apply a pulse to move the qubit's state halfway between
        # the 0 state and the 1 state
        program += RX(math.pi / 2, 0)
        program += RX(math.pi / 2, 1)
        program += RX(math.pi / 2, 2)
        # Add measurement instructions to measure the qubits and record the result
        # into the respective bit in the readout register
        program += MEASURE(0, readout[0])
        program += MEASURE(1, readout[1])
        program += MEASURE(2, readout[2])

    # This tells the program how many times to run the above sequence
    program.wrap_in_numshots_loop(shots)

    # Get the quantum computer we want to run our experiment on
    qc = get_qc(device_name)

    # Compile the program, specific to which quantum computer we are using
    compiled_program = qc.compile(program)

    # Run the program and get the shots x 3 array of results
    results = qc.run(compiled_program)
    # Print the results. We expect to see (shots x 3) random 0's and 1's
    print(f"Your{' virtual' if isinstance(qc.qam, QVM) else ''} quantum "
          f"computer, {device_name}, greets you with:\n", results)

In [4]:
hello_qmi()

Your virtual quantum computer, 9q-generic-qvm, greets you with:
 [[1 0 0]
 [0 1 0]
 [1 1 1]
 [0 0 0]
 [1 0 0]]


Congratulations! You just ran your first program against the QVM simulator.

## Run against the real QPU
To run against a real quantum computer, you only need to change the device argument to the `hello_qmi()` program. What QPU devices are available? We can use pyQuil to get a list.

In [5]:
# Query available devices (QPUs)
from pyquil.api._devices import list_devices, list_lattices

In [6]:
device_names = list_devices()  # Available devices are subject to change.
lattice_names = list(list_lattices().keys())  # Available lattices are subject to change.
print(f"Available devices: {device_names}.\n")
print(f"Available lattices: {lattice_names}.")

Available devices: ['Aspen-4', 'Aspen-7'].

Available lattices: ['Aspen-7-28Q-A', 'Aspen-7-25Q-B', 'Aspen-7-16Q-B', 'Aspen-7-15Q-B', 'Aspen-7-14Q-B', 'Aspen-7-13Q-B', 'Aspen-4-13Q-E', 'Aspen-7-12Q-B', 'Aspen-7-11Q-B', 'Aspen-7-10Q-B', 'Aspen-7-9Q-B', 'Aspen-4-9Q-E', 'Aspen-7-8Q-B', 'Aspen-4-8Q-E', 'Aspen-7-7Q-B', 'Aspen-4-7Q-E', 'Aspen-7-6Q-B', 'Aspen-4-6Q-E', 'Aspen-7-5Q-B', 'Aspen-4-5Q-E', 'Aspen-7-4Q-B', 'Aspen-7-4Q-C', 'Aspen-4-4Q-E', 'Aspen-4-4Q-D', 'Aspen-4-4Q-A', 'Aspen-4-3Q-A', 'Aspen-7-3Q-B', 'Aspen-4-3Q-E', 'Aspen-4-3Q-D', 'Aspen-4-2Q-C', 'Aspen-7-2Q-B', 'Aspen-4-2Q-A'].


### A quick note on lattices
What are these lattices we just queried? They're subsets of qubits on a QPU, so you can use just the qubits you need. QCS not only makes QPU devices available, but also allows users to target subsets of qubits on each QPU, which we call _lattices_.

So let's pick a lattice to target and pass it to the `hello_qmi` function. If you look in `hello_qmi`, you'll see that it's targeting qubits `[0, 1, 2]`. Let's find a lattice that includes these qubits.

In [7]:
lattice_name = lattice_names[0]
hello_qmi(lattice_name)  # ALERT: This will probably result in an error. Please see below.

ERROR - Failed to engage: Your authorization token is missing. Follow the steps at https://qcs.rigetti.com/auth/token to obtain a token.


UserMessageError: Failed to engage: Your authorization token is missing. Follow the steps at https://qcs.rigetti.com/auth/token to obtain a token.

### Hey, why didn't that work?
Did you get an error above? If so, **it's because you haven't yet booked time on the QPU**. QCS makes its QPUs available by reservation. You can book reservations via the [QCS Dashboard](https://qcs.rigetti.com/dashboard), or via the `qcs` command, which comes pre-installed in your QMI. If you've `ssh`'d into your QMI, you can type the following:

```
qcs reserve --lattice Aspen-1-2Q-B
```

Give it a try! Once your reservation begins, you can then re-run the above cell on the lattice you've booked, and get results from our QPU.

## Where to go from here

This is just the beginning of your journey with QCS. Next, feel free to take a look under the hood at some of the QCS features that unlock the full potential of the platform for fast hybrid programming. These can be found in the other pre-loaded example notebooks. Or try writing some new notebooks of your own.

As a gentle reminder, we recommend you learn how to `ssh` into your QMI, and move files to and from it via `scp` by following the docs [here](https://www.rigetti.com/qcs/docs/getting-started-with-your-qmi#qmi). If you have any trouble getting things set up, please reach out to support@rigetti.com.

Happy quantum programming.

*-- The QCS Team*